In this notebook, we will calculate features using Trotterization with IBM's QPU.

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
# This is necessary only when running the notebook in the docker container
%cd fourier_learning_ibm/

/root/workdir/fourier_learning_ibm


/usr/local/lib/python3.10/dist-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [3]:
# GPU check
!nvidia-smi -L

GPU 0: NVIDIA A100-PCIE-40GB (UUID: GPU-3fac7503-bb55-35cd-f566-c919eaa55798)
GPU 1: NVIDIA A100-PCIE-40GB (UUID: GPU-bf03abcd-ef36-6733-da57-460c8eb9cab4)
GPU 2: NVIDIA A100-PCIE-40GB (UUID: GPU-bc01f8f3-0939-f2c0-6998-dd49d0360463)
GPU 3: NVIDIA A100-PCIE-40GB (UUID: GPU-b7aeac00-b332-1045-eebc-bb5653ac8402)


In [4]:
import numpy as np
import scipy
import matplotlib.pyplot as plt
import pandas as pd
import os
from heisenberg import (
    HeisenbergModel,
    get_graph,
    get_positions,
    # get_initial_layout,
    # get_prob0,
)
from setup import setup_backend
from helper_features import run_job, get_features
from qiskit import transpile
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2 as Sampler, Batch
from qiskit_aer import AerSimulator
from qiskit_aer.noise import NoiseModel
import pickle
import networkx as nx
from datetime import datetime, timezone
import json
import math

In [5]:
import warnings

warnings.filterwarnings("ignore")

### Config backend

In [6]:
# density matrix simulator
backend_qpu, backend_dm_noiseless, backend_dm_noisy = setup_backend(
    qpu_name="ibm_marrakesh", device="GPU", method="density_matrix"
)

print()

# matrix product state simulator
backend_qpu, backend_mps_noiseless, backend_mps_noisy = setup_backend(
    qpu_name="ibm_marrakesh", device="GPU", method="matrix_product_state"
)

Using backend QPU: <IBMBackend('ibm_marrakesh')>

NoiseModel:
  Basis gates: ['cz', 'delay', 'for_loop', 'id', 'if_else', 'measure', 'reset', 'rz', 'switch_case', 'sx', 'x']
  Instructions with noise: ['id', 'measure', 'sx', 'cz', 'x', 'reset']
  Qubits with noise: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155]
  Specific qubit errors: [('id', (0,)), ('id', (1,)), ('id', (2

### Prepare parameters and load dataset

In [7]:
n_qubits = 52

C = 3
n_samples = 55

if n_qubits <= 50:  # n_qubits <= 50
    threshold = 0.6
    n_shots = 1000

    # Load dataset (orient="records")
    df_n_step = pd.read_json(
        f"results/get_nstep/{n_qubits}Q/threshold{threshold:.2f}/optimal_n_step.json",
        orient="records",
    )
    display(df_n_step)

    # Note that K = n_features-1.
    # n_features counts only the number of real features, also the one of t=0, i.e., constant term.
    # The number of all features, including imaginary numbers, is 2K+1.
    n_features = len(df_n_step)

    times = df_n_step["t"].values
    n_step_array = df_n_step["n_step"].values

else:  # n_qubits > 50
    threshold = None
    n_features = 20
    n_shots = 100
    # 間隔 np.pi/C で等間隔な点を n_features 個生成
    times = np.linspace(0, np.pi / C * (n_features - 1), n_features)

    # n_qubits > 50 の場合は、n_step は 1 から 7 の整数値を取る。
    # まず 1から7までの20個の等間隔の数値を生成
    n_step_array = np.linspace(1, 7, 20)
    # 各値を四捨五入して整数に変換
    n_step_array = [round(x) for x in n_step_array]

print(f"n_qubits: {n_qubits}")
print(f"C: {C}")
print(f"threshold: {threshold}")
print(f"n_samples: {n_samples}")
print(f"n_shots: {n_shots}")
print(f"n_features: {n_features}")
print(f"times: {times}")
print(f"n_step_array: {n_step_array}")

# Load dataset (orient="records")
df_dataset = pd.read_json(
    f"results/get_dataset/{n_qubits}Q/dataset.json",
    orient="records",
)
all_Js = df_dataset["Js"].values
all_expected_values = df_dataset["expected_value"].values
n_samples = len(all_Js)
display(df_dataset)

n_qubits: 52
C: 3
threshold: None
n_samples: 55
n_shots: 100
n_features: 20
times: [ 0.          1.04719755  2.0943951   3.14159265  4.1887902   5.23598776
  6.28318531  7.33038286  8.37758041  9.42477796 10.47197551 11.51917306
 12.56637061 13.61356817 14.66076572 15.70796327 16.75516082 17.80235837
 18.84955592 19.89675347]
n_step_array: [1, 1, 2, 2, 2, 3, 3, 3, 4, 4, 4, 4, 5, 5, 5, 6, 6, 6, 7, 7]


,sample,Js,expected_value,expected_value_diag,diff
0,0,"[0.021581075300000002, -0.004814892800000001, ...",0.874333,NaN,NaN
1,1,"[-0.0424831577, 0.024743239900000002, 0.014216...",1.264982,NaN,NaN
2,2,"[-0.019849499200000002, 0.039776682300000005, ...",1.043115,NaN,NaN
3,3,"[0.0038696408, -0.0280307067, 0.01233447190000...",0.878768,NaN,NaN
4,4,"[-0.038509265, -0.021418483500000002, -0.02916...",0.920255,NaN,NaN
5,5,"[0.0169817178, 0.021851908, -0.027246614500000...",1.089466,NaN,NaN
6,6,"[-0.0100976512, 0.0122639897, -0.0205218139, -...",0.917398,NaN,NaN
7,7,"[0.0228687258, -0.0005996629, 0.00761107880000...",0.887918,NaN,NaN
8,8,"[-0.024747151000000002, 0.0313821979, 0.010417...",1.136008,NaN,NaN
9,9,"[0.0145011702, 0.0076980293, -0.0377251604, 0....",1.025680,NaN,NaN


# Create Fourier feature

## Trotter simulation

In [9]:
config = {
    "n_qubits": n_qubits,
    "n_samples": n_samples,
    "n_shots": n_shots,
    "n_features": n_features,
    "times": times,
    "all_Js": all_Js,
    "n_step_array": n_step_array,
    "backend": backend_qpu,  # This is important!!
    "all_expected_values": all_expected_values,
}

In [10]:
# For QPU, we need batch instance and job_ids instead of job instances
batch, _, job_ids = run_job(config, backend_qpu, sim_type="qpu")

In [11]:
path = f"results/fourier_feature_qpu/{n_qubits}Q"

In [12]:
# Save job_ids
with open(f"{path}/job_ids.txt", "w") as f:
    for job_id in job_ids:
        f.write(f"{job_id}\n")

In [13]:
batch.status()

'Pending'

In [12]:
# Load job_ids
with open(f"{path}/job_ids.txt", "r") as f:
    job_ids = [job_id.strip() for job_id in f.readlines()]

# fetch successful jobs
jobs = []
error_count = 0
for i in range(n_samples):
    print(f"Checking job {i}/{n_samples}")
    job_id = job_ids[i]
    job = backend_qpu.service.job(job_id)

    if job.status() == "DONE":
        jobs.append(job)
    else:
        error_count += 1
        print(f"Status: {job.status()}")

Checking job 0/55
Checking job 1/55
Checking job 2/55
Checking job 3/55
Checking job 4/55
Checking job 5/55
Checking job 6/55
Checking job 7/55
Checking job 8/55
Checking job 9/55
Checking job 10/55
Checking job 11/55
Checking job 12/55
Checking job 13/55
Checking job 14/55
Checking job 15/55
Checking job 16/55
Checking job 17/55
Checking job 18/55
Checking job 19/55
Checking job 20/55
Checking job 21/55
Checking job 22/55
Checking job 23/55
Checking job 24/55
Checking job 25/55
Checking job 26/55
Checking job 27/55
Checking job 28/55
Checking job 29/55
Checking job 30/55
Checking job 31/55
Checking job 32/55
Checking job 33/55
Checking job 34/55
Checking job 35/55
Checking job 36/55
Checking job 37/55
Checking job 38/55
Checking job 39/55
Checking job 40/55
Checking job 41/55
Checking job 42/55
Checking job 43/55
Checking job 44/55
Checking job 45/55
Checking job 46/55
Checking job 47/55
Checking job 48/55
Checking job 49/55
Checking job 50/55
Checking job 51/55
Checking job 52/55
Che

In [13]:
features_df = get_features(config, jobs)
display(features_df)

Post-processing sample 0/55
Post-processing sample 1/55
Post-processing sample 2/55
Post-processing sample 3/55
Post-processing sample 4/55
Post-processing sample 5/55
Post-processing sample 6/55
Post-processing sample 7/55
Post-processing sample 8/55
Post-processing sample 9/55
Post-processing sample 10/55
Post-processing sample 11/55
Post-processing sample 12/55
Post-processing sample 13/55
Post-processing sample 14/55
Post-processing sample 15/55
Post-processing sample 16/55
Post-processing sample 17/55
Post-processing sample 18/55
Post-processing sample 19/55
Post-processing sample 20/55
Post-processing sample 21/55
Post-processing sample 22/55
Post-processing sample 23/55
Post-processing sample 24/55
Post-processing sample 25/55
Post-processing sample 26/55
Post-processing sample 27/55
Post-processing sample 28/55
Post-processing sample 29/55
Post-processing sample 30/55
Post-processing sample 31/55
Post-processing sample 32/55
Post-processing sample 33/55
Post-processing sample 3

,sample_id,f_0 Re,f_1 Re,f_1 Im,f_2 Re,f_2 Im,f_3 Re,f_3 Im,f_4 Re,f_4 Im,...,f_7 Im,f_8 Re,f_8 Im,f_9 Re,f_9 Im,f_10 Re,f_10 Im,f_11 Re,f_11 Im,expected_value
0,0,0.894,0.608061,-0.269830,0.393699,-0.335748,0.154253,-0.273774,0.081963,-0.136081,...,-0.050146,0.059641,-0.077923,-0.081465,-0.208085,-0.179955,-0.198950,-0.281172,-0.168140,0.792109
1,1,0.904,0.702251,-0.051360,0.598211,0.035118,0.508561,0.114587,0.336921,0.067705,...,-0.098856,-0.130094,-0.169244,-0.146314,-0.252769,-0.156456,-0.278579,-0.177029,-0.337412,1.102887
2,2,0.876,0.738979,0.059056,0.640933,0.145929,0.527899,0.188826,0.430923,0.157639,...,0.104131,0.182071,0.085880,0.179083,0.108638,0.140444,0.103371,0.127039,0.067831,1.130663
3,3,0.919,0.657926,0.043011,0.407268,0.252442,0.159709,0.168884,-0.013340,0.050597,...,0.052190,0.025068,0.052389,0.083327,0.023763,0.176146,-0.002372,0.221758,0.118829,1.552866
4,4,0.908,0.728017,-0.068817,0.644951,0.097325,0.540864,0.180409,0.438869,0.130442,...,0.206299,0.090603,0.197183,-0.072240,0.105837,-0.076163,0.064808,-0.083124,0.044770,1.149036
5,5,0.903,0.686750,0.204094,0.541424,0.277857,0.324335,0.319512,0.140685,0.193230,...,-0.143934,0.363322,-0.056195,0.363176,0.137699,0.269235,0.210220,0.149341,0.221678,1.332599
6,6,0.916,0.631380,-0.065393,0.414109,-0.072627,0.137418,-0.026801,0.003400,0.020819,...,-0.148042,-0.195260,-0.152772,-0.202358,0.072864,-0.107078,0.053844,-0.009398,0.017311,1.114135
7,7,0.889,0.645722,-0.090909,0.507925,-0.092032,0.260203,-0.004854,0.074223,0.004002,...,0.031565,-0.004106,0.003484,0.001832,-0.016451,-0.097754,-0.008371,-0.132881,0.032596,1.034106
8,8,0.894,0.642764,0.133629,0.503196,0.263171,0.283667,0.310536,0.089760,0.211724,...,-0.082219,0.267951,-0.022345,0.289722,0.040576,0.233645,0.086552,0.242579,0.126824,1.421492
9,9,0.908,0.543202,-0.408726,0.362215,-0.554823,0.036693,-0.594970,-0.208842,-0.505625,...,0.148678,-0.160398,0.207712,-0.019776,0.245695,0.061629,0.223801,0.186722,0.143832,0.634733
